# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#512 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846842727876                   -0.70    4.2
  2   -7.852322843061       -2.26       -1.53    1.0
  3   -7.852646088990       -3.49       -2.52    3.2
  4   -7.852646678900       -6.23       -3.36    2.5
  5   -7.852646686053       -8.15       -4.82    1.0
  6   -7.852646686726       -9.17       -5.24    3.8
  7   -7.852646686730      -11.39       -6.09    1.0
  8   -7.852646686730      -12.60       -7.25    2.8
  9   -7.852646686730   +    -Inf       -7.59    2.2
 10   -7.852646686730      -14.75       -8.40    2.0
 11   -7.852646686730   +  -14.57       -9.56    2.0
 12   -7.852646686730      -15.05       -9.37    1.2
 13   -7.852646686730   +    -Inf      -10.56    1.8
 14   -7.852646686730   +    -Inf      -10.68    2.0
 15   -7.852646686730   +    -Inf      -11.47    1.0
 16   -7.852646686730   +  -14.57      -11.63    1.8
 17   -7.852646686730      -14.57      -11.90 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846819379907                   -0.70           4.2
  2   -7.852528003848       -2.24       -1.62   0.80    5.0
  3   -7.852609688719       -4.09       -2.78   0.80    5.0
  4   -7.852646571415       -4.43       -3.57   0.80    7.2
  5   -7.852646683368       -6.95       -4.55   0.80    8.8
  6   -7.852646686679       -8.48       -5.09   0.80   10.5
  7   -7.852646686724      -10.34       -6.12   0.80   11.2
  8   -7.852646686730      -11.27       -7.13   0.80   13.5
  9   -7.852646686730      -13.32       -7.77   0.80   15.0
 10   -7.852646686730      -15.05       -9.38   0.80   16.8
 11   -7.852646686730      -15.05       -9.29   0.80   19.0
 12   -7.852646686730   +  -14.45      -10.19   0.80   19.8
 13   -7.852646686730      -14.27      -11.67   0.80   21.0
 14   -7.852646686730   +  -14.45      -12.33   0.80   23.5


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.425348e+01     2.918826e+00
 * time: 0.4231750965118408
     1     1.268181e+00     1.938401e+00
 * time: 0.619391918182373
     2    -1.734181e+00     2.068951e+00
 * time: 0.6431360244750977
     3    -3.725216e+00     1.999793e+00
 * time: 0.6774039268493652
     4    -5.034507e+00     2.027656e+00
 * time: 0.7114109992980957
     5    -6.687103e+00     1.604574e+00
 * time: 0.7459530830383301
     6    -7.398493e+00     7.650012e-01
 * time: 0.7798011302947998
     7    -7.648230e+00     4.028789e-01
 * time: 0.8034560680389404
     8    -7.754205e+00     1.161407e-01
 * time: 0.8271400928497314
     9    -7.787118e+00     1.313925e-01
 * time: 0.8508830070495605
    10    -7.808108e+00     8.301570e-02
 * time: 0.8746471405029297
    11    -7.823295e+00     9.649904e-02
 * time: 0.8989090919494629
    12    -7.840840e+00     5.387239e-02
 * time: 0.9229109287261963
    13    -7.849539e+00     2.541474e-02
 * time: 0.94677400588

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846889101620                   -0.70    4.2
  2   -7.852328456917       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686711                   -2.54
  2   -7.852646686730      -10.72       -5.93
  3   -7.852646686730   +  -14.75      -12.66
|ρ_newton - ρ_scf|  = 2.6574363024722467e-13
|ρ_newton - ρ_scfv| = 5.4355464968945945e-14
|ρ_newton - ρ_dm|   = 3.5514212961804796e-10
